<a href="https://colab.research.google.com/github/danielmartincampos/FIA-TP5/blob/main/TP5_PARTE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalamos librerías

In [ ]:
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("caret") #para el particionado del df
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘iterators’, ‘Rcpp’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Importamos librerías

In [ ]:
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice

Thank you for using fastDummies!

To acknowledge our work, please cite the package:

Kaplan, J. & Schlegel, B. (2023). fastDummies: Fast Creation of Dummy (Binary) Columns and Rows from Categorical Variables. Version 1.7.1. URL: https://github.com/jacobkap/fastDummies, https://jacobkap.github.io/fastDummies/.



In [ ]:
df_credit_card <- read.csv("/content/Credit_card_limpio.csv")

In [ ]:
head(df_credit_card)

,X,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,⋯,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Family_Members,label,Birthday_date,age,Employed
,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>
1,1,5008827,M,Y,Y,0,180000,Pensioner,Higher education,Married,⋯,365243,1,0,0,0,2,1,1972-06-29,51,n
2,2,5008865,F,Y,Y,2,135000,Working,Secondary / secondary special,Married,⋯,-3173,1,0,0,0,4,0,1980-09-26,43,y
3,3,5008889,F,N,Y,0,247500,Commercial associate,Higher education,Separated,⋯,-1347,1,0,0,0,1,0,1977-04-20,46,y
4,4,5009000,M,Y,Y,0,157500,Working,Secondary / secondary special,Married,⋯,-828,1,0,0,0,2,0,1996-09-16,27,y
5,5,5009023,F,N,Y,2,216000,State servant,Higher education,Married,⋯,-3112,1,0,0,1,4,0,1981-08-09,42,y
6,6,5009053,F,N,Y,2,202500,Working,Higher education,Married,⋯,-2289,1,0,0,1,4,0,1993-10-12,30,y


Dejamos las columnas con las que vamos a trabajar

In [ ]:
colnames(df_credit_card)

[1] "X"              "Ind_ID"         "GENDER"         "Car_Owner"     
 [5] "Propert_Owner"  "CHILDREN"       "Annual_income"  "Type_Income"   
 [9] "EDUCATION"      "Marital_status" "Housing_type"   "Employed_days" 
[13] "Mobile_phone"   "Work_Phone"     "Phone"          "EMAIL_ID"      
[17] "Family_Members" "label"          "Birthday_date"  "age"           
[21] "Employed"

In [ ]:
columnas_a_usar <- c('GENDER','Car_Owner','Propert_Owner','CHILDREN','Annual_income','Family_Members',
  'label','age','Employed','EDUCATION', 'Marital_status', 'Type_Income', 'Housing_type')

# Creamos un subconjunto con las columnas especificadas
df_credit_card <- df_credit_card[ , columnas_a_usar]

In [ ]:
str(df_credit_card) #Verificamos que haya quedado la estructura que deseamos

'data.frame':	1496 obs. of  13 variables:
 $ GENDER        : chr  "M" "F" "F" "M" ...
 $ Car_Owner     : chr  "Y" "Y" "N" "Y" ...
 $ Propert_Owner : chr  "Y" "Y" "Y" "Y" ...
 $ CHILDREN      : int  0 2 0 0 2 2 1 0 0 0 ...
 $ Annual_income : num  180000 135000 247500 157500 216000 ...
 $ Family_Members: int  2 4 1 2 4 4 3 1 2 2 ...
 $ label         : int  1 0 0 0 0 0 0 0 0 0 ...
 $ age           : int  51 43 46 27 42 30 48 43 43 57 ...
 $ Employed      : chr  "n" "y" "y" "y" ...
 $ EDUCATION     : chr  "Higher education" "Secondary / secondary special" "Higher education" "Secondary / secondary special" ...
 $ Marital_status: chr  "Married" "Married" "Separated" "Married" ...
 $ Type_Income   : chr  "Pensioner" "Working" "Commercial associate" "Working" ...
 $ Housing_type  : chr  "House / apartment" "House / apartment" "Rented apartment" "House / apartment" ...


In [ ]:
df_credit_card <- df_credit_card %>% select(-label, everything(), label) #Movemos la columna de la etiqueta al final

In [ ]:
unique(df_credit_card$EDUCATION)

[1] "Higher education"              "Secondary / secondary special"
[3] "Incomplete higher"             "Lower secondary"              
[5] "Academic degree"

In [ ]:
df_credit_card$label <- factor(df_credit_card$label)

Para SVM tenemos que castear algunas columnas con variables categórícas, es decir, convertims cada categoría en una columna con valores 0 o 1 dependiendo si se cumple o no la categoría.


In [ ]:
columnas_a_castear <- c('EDUCATION', 'Marital_status', 'Type_Income', 'Housing_type')

df_credit_cardSVM<-melt(df_credit_card,measure.vars = columnas_a_castear)
# Utilizamos fastDummies para convertir las columnas en filas
df_credit_cardSVM <- dcast(df_credit_cardSVM,...~value,fun.aggregate = length,fill = 0)

In [ ]:
colnames(df_credit_cardSVM)

[1] "GENDER"                        "Car_Owner"                    
 [3] "Propert_Owner"                 "CHILDREN"                     
 [5] "Annual_income"                 "Family_Members"               
 [7] "age"                           "Employed"                     
 [9] "label"                         "variable"                     
[11] "Academic degree"               "Civil marriage"               
[13] "Co-op apartment"               "Commercial associate"         
[15] "Higher education"              "House / apartment"            
[17] "Incomplete higher"             "Lower secondary"              
[19] "Married"                       "Municipal apartment"          
[21] "Office apartment"              "Pensioner"                    
[23] "Rented apartment"              "Secondary / secondary special"
[25] "Separated"                     "Single / not married"         
[27] "State servant"                 "Widow"                        
[29] "With parents"                  "Working"

In [ ]:
str(df_credit_cardSVM)

'data.frame':	5112 obs. of  30 variables:
 $ GENDER                       : chr  "F" "F" "F" "F" ...
 $ Car_Owner                    : chr  "N" "N" "N" "N" ...
 $ Propert_Owner                : chr  "N" "N" "N" "N" ...
 $ CHILDREN                     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Annual_income                : num  33750 33750 33750 33750 45000 ...
 $ Family_Members               : int  2 2 2 2 1 1 1 1 2 2 ...
 $ age                          : int  52 52 52 52 39 39 39 39 38 38 ...
 $ Employed                     : chr  "n" "n" "n" "n" ...
 $ label                        : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ variable                     : Factor w/ 4 levels "EDUCATION","Marital_status",..: 1 2 3 4 1 2 3 4 1 2 ...
 $ Academic degree              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Civil marriage               : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Co-op apartment              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Commercial associate         : num  0 0 0 0 0 0 0 0 0 0 ...
 $

In [ ]:
df_credit_cardSVM<-subset(df_credit_cardSVM,select = -variable)

In [ ]:
df_credit_cardSVM<-df_credit_cardSVM %>% select(-label,everything(),label)

In [ ]:
str(df_credit_cardSVM)

'data.frame':	5112 obs. of  29 variables:
 $ GENDER                       : chr  "F" "F" "F" "F" ...
 $ Car_Owner                    : chr  "N" "N" "N" "N" ...
 $ Propert_Owner                : chr  "N" "N" "N" "N" ...
 $ CHILDREN                     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Annual_income                : num  33750 33750 33750 33750 45000 ...
 $ Family_Members               : int  2 2 2 2 1 1 1 1 2 2 ...
 $ age                          : int  52 52 52 52 39 39 39 39 38 38 ...
 $ Employed                     : chr  "n" "n" "n" "n" ...
 $ Academic degree              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Civil marriage               : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Co-op apartment              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Commercial associate         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Higher education             : num  1 0 0 0 0 0 0 0 0 0 ...
 $ House / apartment            : num  0 0 0 1 0 0 0 1 0 0 ...
 $ Incomplete higher            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Lower 

In [ ]:
# Eliminamos las columnas en df_credit_cardSVM
df_credit_cardSVM <- df_credit_cardSVM[, !(names(df_credit_cardSVM) %in% columnas_a_castear)]

In [ ]:
df_credit_cardSVM <- df_credit_cardSVM %>% select(-label, everything(), label) #Movemos la columna de la etiqueta al final

In [ ]:
table(df_credit_cardSVM$label)


   0    1 
4660  452 

# Solucionamos el desbalanceo

Opción 1, elegir 160 filas aleatorias con label = 0

In [ ]:
# Creamos un subconjunto de 160 filas con label = 0 al azar
df_label_0 <- df_credit_card[df_credit_card$label == 0, ] # Creamos un df con las filas con etiqueta 0
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 160), ]

# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_card[df_credit_card$label == 1, ]

# Combinamos los subconjuntos en un DataFrame balanceado
df_balanceado160 <- rbind(sampled_df_label_0, df_label_1)

In [ ]:
table(df_balanceado160$label)


  0   1 
160 160 

Opción 2, duplicamos las flas con label = 1

In [ ]:
# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_card[df_credit_card$label == 1, ]

# Duplicamos las filas con label = 1
duplicados_label_1 <- rbind(df_label_1, df_label_1)

# Creamos un subconjunto de 320 filas con label = 0 al azar
df_label_0 <- df_credit_card[df_credit_card$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 320), ]

# Combinamos los DataFrames de 320 filas obtenidos
df_balanceado320 <- rbind(duplicados_label_1, sampled_df_label_0)

In [ ]:
str(df_balanceado320)

'data.frame':	640 obs. of  13 variables:
 $ GENDER        : chr  "M" "F" "F" "F" ...
 $ Car_Owner     : chr  "Y" "Y" "Y" "Y" ...
 $ Propert_Owner : chr  "Y" "N" "N" "N" ...
 $ CHILDREN      : int  0 0 0 0 0 1 1 1 0 0 ...
 $ Annual_income : num  180000 315000 315000 315000 180000 450000 450000 450000 90000 90000 ...
 $ Family_Members: int  2 2 2 2 2 3 3 1 2 2 ...
 $ age           : int  51 37 37 37 60 49 49 49 51 51 ...
 $ Employed      : chr  "n" "y" "y" "y" ...
 $ EDUCATION     : chr  "Higher education" "Higher education" "Higher education" "Higher education" ...
 $ Marital_status: chr  "Married" "Married" "Married" "Married" ...
 $ Type_Income   : chr  "Pensioner" "Commercial associate" "Commercial associate" "Commercial associate" ...
 $ Housing_type  : chr  "House / apartment" "House / apartment" "House / apartment" "House / apartment" ...
 $ label         : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...


In [ ]:
table(df_balanceado320$label)


  0   1 
320 320 

# Comenzamos con SVM
Creamos los df balanceados con el df resultante después del casteo

Opción 1 TRABAJAMOS CON EL DATASET BALANCEADO DE 160 DATOS POR CLASE

In [ ]:
# Creamos un subconjunto de 160 filas con label = 0 al azar
df_label_0 <- df_credit_cardSVM[df_credit_cardSVM$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 160), ]

# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_cardSVM[df_credit_cardSVM$label == 1, ]

# Combinamos los subconjuntos en un DataFrame balanceado
df_balanceado160SVM <- rbind(sampled_df_label_0, df_label_1)

In [ ]:
set.seed(2023) #seteamos la semilla para que el experimento sea reproducible
t.ids <- createDataPartition(df_balanceado160SVM$label, p =0.7, list=F) #particionamos el df en 70% para entrenamiento y 30% para prueba
modelo <- svm(label ~ ., data = df_balanceado160SVM[t.ids, ]) #entrenamiento
prediccion <- predict(modelo, df_balanceado160SVM[-t.ids, ]) #prueba
matriz_confusion <- table(df_balanceado160SVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho')) #creamos una matriz de confusión

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘X.Academic.degree.’ constant. Cannot scale data.”


In [ ]:
table(df_balanceado160$label)


  0   1 
160 160 

In [ ]:
matriz_confusion

    Predicho
Real   0   1
   0  13  35
   1   3 132

In [ ]:
sum(prediccion != df_balanceado160SVM$label[-t.ids])/length(df_balanceado160SVM$label[-t.ids]) #Error calculado como (total de filas mal clasificadas/total de filas)

[1] 0.2076503

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion) #Error calculado usando la matriz de confusión (equivalente a la anterior)VER


[1] 0.2076503

In [ ]:
# Calculamos las métricas de precisión y sensibilidad
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])
print(precision)
print(sensibilidad)

[1] 0.8087432
[1] 0.9925926


Opción 2 TRABAJAMOS CON EL DATASET BALANCEADO DE 320 DATOS POR CLASE

In [ ]:
# Creamos un subconjunto con las filas con label = 1
df_label_1 <- df_credit_cardSVM[df_credit_cardSVM$label == 1, ]

# Duplicamos las filas con label = 1
duplicados_label_1 <- rbind(df_label_1, df_label_1)

# Creamos un subconjunto de 1320 filas con label = 0 al azar
df_label_0 <- df_credit_cardSVM[df_credit_cardSVM$label == 0, ]
sampled_df_label_0 <- df_label_0[sample(nrow(df_label_0), 320), ]

# Combinamos los DataFrames de 320 filas obtenidos
df_balanceado320SVM <- rbind(duplicados_label_1, sampled_df_label_0)

In [ ]:
set.seed(2023)
t.ids <- createDataPartition(df_balanceado320SVM$label, p =0.7, list=F)
modelo <- svm(label ~ ., data = df_balanceado320SVM[t.ids, ])
prediccion <- predict(modelo, df_balanceado320SVM[-t.ids, ])
matriz_confusion <- table(df_balanceado320SVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
“Variable(s) ‘X.Academic.degree.’ constant. Cannot scale data.”


In [ ]:
print(matriz_confusion)

    Predicho
Real   0   1
   0  57  39
   1   5 266


In [ ]:
sum(prediccion != df_balanceado320SVM$label[-t.ids])/length(df_balanceado320SVM$label[-t.ids])

[1] 0.119891

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.119891

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.880109
[1] 0.9815498


Opción 3- TRABAJAMOS CON EL DATASET DESBALANCEADO

In [ ]:
set.seed(2023)
t.ids <- createDataPartition(df_credit_cardSVM$label, p =0.7, list=F)
modelo <- svm(label ~ ., data = df_credit_cardSVM[t.ids, ])
prediccion <- predict(modelo, df_credit_cardSVM[-t.ids, ])
matriz_confusion <- table(df_credit_cardSVM$label[-t.ids], prediccion, dnn = c('Real', 'Predicho'))

In [ ]:
print(matriz_confusion)

    Predicho
Real    0    1
   0 1398    0
   1  135    0


In [ ]:
sum(prediccion != df_credit_cardSVM$label[-t.ids])/length(df_credit_cardSVM$label[-t.ids])

[1] 0.08806262

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.08806262

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.9119374
[1] 0


# Random Forest
Trabajamos con las columnas de variables categóricas sin castear

TRABAJAMOS CON EL DATASET BALANCEADO DE 160 DATOS POR CLASE

In [ ]:
set.seed(2023)
t.ids <- createDataPartition(df_balanceado160$label, p =0.7, list=F)
modelo_rf <- randomForest(label ~ ., data = df_balanceado160[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_balanceado160[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_balanceado160$label[-t.ids], dnn = c('Real' ,'Predicho'))

In [ ]:
matriz_confusion

    Predicho
Real  0  1
   0 18 15
   1 30 33

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.3125

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.6875
[1] 0.6875


TRABAJAMOS CON EL DATASET BALANCEADO DE 320 DATOS POR CLASE

In [ ]:
set.seed(2023)
t.ids <- createDataPartition(df_balanceado320$label, p =0.7, list=F)
modelo_rf <- randomForest(label ~ ., data = df_balanceado320[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_balanceado320[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_balanceado320$label[-t.ids], dnn = c('Real' ,'Predicho'))

In [ ]:
matriz_confusion

    Predicho
Real  0  1
   0 66 16
   1 30 80

In [ ]:
sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.7604167

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.7604167
[1] 0.7272727


TRABAJADMOS CON EL DATA SET ORIGINAL EL CUAL ESTA DESBALANCEADO

In [ ]:
set.seed(2023)
t.ids <- createDataPartition(df_credit_card$label, p =0.7, list=F)
modelo_rf <- randomForest(label ~ ., data = df_credit_card[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_credit_card[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_credit_card$label[-t.ids], dnn = c('Real' ,'Predicho'))

In [ ]:
matriz_confusion

    Predicho
Real   0   1
   0 394  40
   1   6   8

In [ ]:
1-sum(diag(matriz_confusion)) / sum(matriz_confusion)

[1] 0.1026786

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.8973214
[1] 0.5714286
